# Simple image classification example

이번 예제는 간단한 CNN을 이용해서 image를 분류해 보는 과정입니다. 
Tensorflow 같은 경우 tutorial이 잘 마련되어 있고, mnist data (숫자 0부터 9까지의 손글씨 그림)를 구분하는 예제 같은 경우 대부분 별 어려움 없이 따라해 보실 수 있을 것입니다. 

그러나, 막상 본인의 데이터를 가지고 비슷한 것을 해보려고 할 때 프로그래밍이 익숙하지 않으신 분들은 대부분 여기에서 어려움을 겪게 되는 것 같습니다. 

그래서 tensorflow 홈페이지에 나와 있는 deep learning을 이용한 MNIST data (28X28 images)를 구분하는 예제를 수정하여 제가 가지고 있는 256x256 images에 적용하는 과정을 정리해 보았습니다.

Tensorflow Deep MNIST example
https://www.tensorflow.org/get_started/mnist/pros

본 예제에 필요한 외부 라이브러리는 skimage, numpy입니다. 


In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

위의 두 줄은 MNIST data를 웹으로부터 다운로드하고 불러오는 과정입니다. 
실제 여기서는 연산에 사용하지 않을 것이지만, 
어떠한 형태로 본인의 데이터를 준비해야 되는 지 참고할 수 있도록 넣어 보았습니다.
mnist.을 입력하고 tab을 누르면 mnist가 가지고 있는 하위 구조가 표시됩니다. 
mnist.train.images.shape를 해보면 train에 사용된 image data가 55000개가 784개의 성분을 가지는 vector의 형태로 저장된 것을 확인할 수 있습니다. 이는 28x28의 2D image를 1D로 배열된 데이터이기 때문입니다.

In [ ]:
mnist.train.images.shape

labe같은 경우는 10개의 성분을 가지는 vector로 되어 있음을 알 수 있습니다.

In [ ]:
mnist.train.labels.shape

실제로 어떻게 되어 있는 지 하나를 가져와서 보면
나머지는 다 0이고 9번째 성분만 1인 벡터임을 알 수 있습니다. 
아마도 8이라는 숫자를 이런 규칙으로 표현했을 것입니다.


In [ ]:
tmplabel = mnist.train.labels[1111]
tmplabel

결국 이 예제에서의 label은 만약 class가 4개라면 각각 아래처럼 준비하면 되겠다는 것을 알 수 있습니다.
[1, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 1]

아래부터가 tensorflow를 이용한 저희의 예제의 시작입니다.

In [ ]:
import tensorflow as tf

제가 준비한 데이터는 각각 256장의 사각형과 삼각형 그림입니다. 
그림은 png로 저장되어 있고, 256x256 사이즈로 저장되어 있습니다. 
(별로 좋은 데이터는 아닙니다. 256개의 그림이 사실상 같은 삼각형/사각형입니다. 단지 코드가 돌아가는 것을 보여주기 위한 그림입니다.)
mydata라는 폴더 안에 사각형은 0이라는 폴더, 삼각형은 1이라는 폴더에 넣었습니다.
본인의 그림으로 테스트를 할 경우 class별로 폴더를 만들어서 정리해 넣으면 됩니다.
예제에 포함되어 있는 mydata 폴더를 참고해주세요.

단, 이 예제에서는 그림은 모두 같은 사이즈의 흑백그림이어야 합니다.
256x256이 아닌 사이즈를 테스트 하려면 코드의 일부를 수정해야 합니다.

아래의 코드는 mydata라는 폴더 안에 있는 png파일들의 list를 준비하면서 각각 속한 하위폴더에 따라 label을 입력하는 과정입니다.

In [ ]:
import os
pathimg = "./mydata"
imgfiles = []
imgtype = []
dn=0
n=0
for dirName, subdirList, fileList in os.walk(pathimg):
    print(dirName, dn)
    for filename in fileList:
        if ".png" in filename.lower():
            imgfiles.append(os.path.join(dirName,filename))
            imgtype.append(dn)
    dn=dn+1
Nclass = dn-1



이제 img, label을 각각 image data, label을 넣기 위해 준비합니다. 
label.shape를 통해 전체 image의 개수와 class의 개수가 제대로 되었는 지 확인합니다.
image size가 256x256이 아니면 256,256을 수정하여야 합니다. 

In [ ]:
import numpy as np
img = np.zeros((len(imgfiles),256,256), dtype=float)
label = np.zeros((len(imgfiles), Nclass), dtype=float)
label.shape

아래는 이미지를 메모리로 불러어고 label vector를 준비하는 과정입니다.
저희 예제처럼 작은 데이터를 가지고 학습을 할 때는 이처럼 memory에 모든 데이터를 불러와서 작업을 할 수 있지만,
다루는 데이터 및 네트워크의 규모가 RAM의 크기로 감당이 되지 않는 큰 경우에는 이런 식으로 작업할 수는 없습니다.

In [ ]:
from skimage.io import imread
for n in range(len(imgfiles)):
    img[n]=imread(imgfiles[n])
    ln = imgtype[n]-1
    label[n,ln] = 1

그 다음 작업은 random하게 90% 정도의 index를 train으로 할당하고, 나머지를 valid로 할당하는 과정입니다.

In [ ]:
train_indices = np.random.choice(len(img),round(len(img)*0.9), replace = False)
valid_indices = np.array(list(set(range(len(img))) - set(train_indices)))

img_train = img[train_indices]
img_valid = img[valid_indices]
label_train = label[train_indices]
label_valid = label[valid_indices]

아래의 network는 tensorflow에서 작성한 것으로 deepnn이라는 함수를 정의해서 CNN의 구조를 만들었습니다.
원래의 예제에서 이미지 사이즈인 28x28에 해당하는 부분들을 256x256에 쓸 수 있도록 숫자들을 변경하였습니다.
만약 image size를 변경할 경우, 아래 세 줄을 찾아서 해당하는 숫자로 바꾸어 주어야 합니다. 2,3번째 행의 64는 256을 4로 나눈 값입니다. 
        x_image = tf.reshape(x, [-1, 256, 256, 1]) # 28, 28 -> 256, 256 image size가 들어가면 됩니다.
        W_fc1 = weight_variable([64 * 64 * 64, 1024]) # image size를 4로 나눈 값을 K라고 하면  k*k*64를 넣으면 됩니다. 다음 행도.
        h_pool2_flat = tf.reshape(h_pool2, [-1, 64 * 64 * 64])

In [ ]:
def deepnn(x):

  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 256, 256, 1]) # 28, 28 -> 256, 256
        
  # First convolutional layer - maps one grayscale image to 32 feature maps.
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

  # Pooling layer - downsamples by 2X.
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

  # Second convolutional layer -- maps 32 feature maps to 64.
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

  # Second pooling layer.
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image ->256x256 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features. -> 64x64x64 feature maps
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([64 * 64 * 64, 1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 64*64*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

  # Map the 1024 features to 10 classes, one for each digit -> N classes
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([1024, Nclass])
        b_fc2 = bias_variable([Nclass])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
  
    return y_conv, keep_prob


아래는 deepnn이라는 메인 network함수에서 자주 사용되는 함수들을 정의해 놓은 것입니다.
def는 함수를 정의한 것으로 실제 실행은 저 함수가 호출되었을 때 일어납니다.

In [ ]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)


학습을 위한 아래와 같은 몇 가지 준비가 더 필요합니다.
image size를 바꿀 경우 256, 256을 본인의 사이즈로 바꾸면 됩니다. 

여기서 중요한 파라미터가 하나 있는데 AdamOptimizer에서 사용하는 1e-3 입니다.
0.001을 표현한 것으로 저 숫자가 작을 수록 학습하는 정도가 바뀝니다. 만약 학습 결과의 변동이 너무 심하거나 하면, 
저 숫자를 1e-4, 1e-5 등으로 줄여줄 필요가 있습니다. 

학습의 결과가 너무 변하지 않는 것 같으면 1e-2로 올려서 테스트 해 볼 수 있습니다.

In [ ]:
# Create the model
x = tf.placeholder(tf.float32, shape = [None, 256, 256])   # 256, 256 -> image size

  # Define loss and optimizer
y_ = tf.placeholder(tf.float32, shape =[None, Nclass])

  # Build the graph for the deep net
y_conv, keep_prob = deepnn(x)

with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv)
  
cross_entropy = tf.reduce_mean(cross_entropy)


with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)


with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  
accuracy = tf.reduce_mean(correct_prediction)



실제 학습과 관련된 연산이 일어나는 것은 아래의 단락을 실행하였을 때입니다.
for i in range(128)의 128은 128번 반복 학습을 하라는 것이고
batch_indices는 전체 데이터 중 한 번 학습을 할 때 랜덤으로 몇 개의 데이터를 추출해서 쓰겠냐는 것인데
메모리사이즈와 데이터크기를 고려해서 정해주어야 합니다. 여기서는 16으로 하였습니다.

작업이 끝나고 마지막으로 valid set에 대한 accuracy를 보여줍니다.

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(32):
        batch_indices = np.random.choice(len(img_train),16, replace=False)
        img_batch = img_train[batch_indices]
        label_batch = label_train[batch_indices]
        if i % 8 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: img_batch, y_: label_batch, keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy)) 
        train_step.run(feed_dict={x: img_batch, y_: label_batch, keep_prob: 0.5})
    print('test accuracy %g' % accuracy.eval(feed_dict={   
        x: img_valid, y_: label_valid, keep_prob: 1.0}))